# Data cleaner
data soucer: https://www.kaggle.com/datasets/carrie1/ecommerce-data

In [1]:
import pandas as pd

In [48]:
raw_data = pd.read_csv('data/raw_data.csv')
raw_data.columns = raw_data.columns.str.lower()
raw_data.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [35]:
print(
    f'Number of unique products: {raw_data.stockcode.nunique()}',
    f'Are there NA values in products: {raw_data.stockcode.isna().any()}',
    f'Are there NA values in unit price: {raw_data.unitprice.isna().any()}',
    f'Are there NA values in quantity: {raw_data.quantity.isna().any()}',
    f'Are there NA values in customerid: {raw_data.customerid.isna().any()}',
    sep='\n'
)

Number of unique products: 4070
Are there NA values in products: False
Are there NA values in unit price: False
Are there NA values in quantity: False
Are there NA values in customerid: True


In [49]:
raw_data = (
    raw_data
    .loc[:, ['customerid', 'quantity', 'unitprice', 'stockcode']]
    .dropna()
    .astype({'customerid': 'int64'})
    .query('quantity > 0')
    .query('unitprice > 0')
    .assign(value = lambda x: x.quantity * x.unitprice)
    .drop(columns=['quantity', 'unitprice'])
)

raw_data.head()

,customerid,stockcode,value
0,17850,85123A,15.30
1,17850,71053,20.34
2,17850,84406B,22.00
3,17850,84029G,20.34
4,17850,84029E,20.34


In [50]:
raw_data.stockcode.astype('category').describe()

count     397884
unique      3665
top       85123A
freq        2035
Name: stockcode, dtype: object

In [67]:
# Pivot the data to get the total value per customer per product category
data = (
    raw_data
    .groupby(['customerid', 'stockcode'])
    .sum()
    .reset_index()
    .pivot(index='customerid', columns='stockcode', values='value')
    .fillna(0)
    .reset_index()
)

data.head()

stockcode,customerid,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
0,12346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0
3,12349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0
4,12350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0


In [65]:
data.describe()

stockcode,customerid,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
count,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,...,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4338.000000,4.338000e+03,4338.000000
mean,15300.408022,0.161261,0.026374,0.009343,0.000749,0.001549,0.001646,0.214454,0.263626,0.411870,...,0.001379,0.000802,0.004299,0.000802,0.038036,1.541263,2.744666,12.397402,6.915629e-07,17.935445
std,1721.808492,3.928573,0.599798,0.195855,0.032727,0.048545,0.054465,6.077177,2.497344,5.957684,...,0.065050,0.052837,0.218659,0.052837,0.820359,64.431572,180.773279,255.364563,2.629152e-05,155.483956
min,12346.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,13813.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
50%,15299.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
75%,16778.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
max,18287.000000,234.600000,28.860000,8.820000,1.950000,1.680000,2.100000,382.500000,59.500000,231.000000,...,3.480000,3.480000,13.920000,3.480000,30.000000,4175.000000,11906.360000,12158.900000,1.000000e-03,8142.750000


In [68]:
data.to_csv('data/data.csv', index=False)

In [69]:
data = pd.read_csv('data/data.csv')
data.head()

,customerid,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
0,12346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0
3,12349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0
4,12350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0
